In [1]:
import pickle
import networkx
from itertools import combinations

creating_pref_attach_graph = False;
creating_pref_attach_reduced_graph = False;
creating_shortest_paths_graph = False;

### LINKS
Tutorial how to use networkX : <a href="https://networkx.github.io/documentation/stable/tutorial.html#accessing-edges-and-neighbors"> networkx.github.io</a> 

In [2]:
with open('base_data/G_california.p', 'rb') as f:
    G = pickle.load(f)

In [3]:
print("Number of nodes : ", G.number_of_nodes())
print("Number of edges : ", G.number_of_edges())
print("\n10 first nodes :\n", "\t"+ "\n\t".join([str(node) for node in list(G.nodes())[:10]]))
print("\n10 first edges :\n", "\t" +"\n\t".join([str(edge) for edge in list(G.edges())[:10]]))

Number of nodes :  9816
Number of edges :  16424

10 first nodes :
 	hortau
	spiritual-gangster
	brit
	seatninja
	bluedata-software
	village-defense
	netherfire-entertainment
	dubuc-motors
	reflektive
	bundled-bliss

10 first edges :
 	('hortau', 'inv_advantage-capital-partners')
	('hortau', 'inv_avrio-capital')
	('spiritual-gangster', 'inv_m3-ventures')
	('brit', 'inv_shervin-pishevar')
	('brit', 'inv_general-catalyst-partners')
	('brit', 'inv_jim-felding')
	('brit', 'inv_cowboy-ventures')
	('brit', 'inv_marissa-mayer')
	('brit', 'inv_jennifer-hyman')
	('brit', 'inv_ff-angel-llc')


In [4]:
invs = [node for node in list(G.nodes()) if node[:4]=='inv_']
vs = [node for node in list(G.nodes()) if node[:4]!='inv_']
n_invs= len(invs)
n_vs = len(vs)
print("Extracted {0} ventures and {1} investors so {2} (/{3}) nodes ".format(len(invs), len(vs), len(invs)+len(vs),  G.number_of_nodes()))

Extracted 5883 ventures and 3933 investors so 9816 (/9816) nodes 


# 2 - Preferential attachement based on degrees

In this part we are interested in preferential attachement, which states that two nodes are likely to be linked when they are already of *high degree* which means, that those nodes are already having a lot of links (edges). This idea states that usually people link to famous people because of the scarcity of information. Nevertheless, there are big limitations to this information in our case, because one start-up can only be invested by a fistful of investors, and a couple of investor for each round.

In the following we show that **preferential attachement of (venture, inv) is 16 times bigger when indeed a link exists between the two stakeholders**. This results might be discussed because, nodes which are connected in the real graph, because of being connected, are likely to have high degree.

To verify more subtily our result, we should get rid of one existing link, calculate the pref attachement between the two nodes, and then suppose that this link is likely to happen, in the same graph as initial except that special edge. To do that, we should just reduce by one the degree of the two nodes in the measure of metric_pref_attach. The result falls back to **14 times larger**

In [5]:

max_dv_dinv_inverse = ((n_invs * n_vs)**(-1))
                       
def metric_pref_attach(node_v, node_inv, G,  max_dv_dinv_inverse):
    return  max_dv_dinv_inverse * G.degree(node_v) * G.degree(node_inv)
                       
def metric_pref_attach_reduced(node_v, node_inv, G,  max_dv_dinv_inverse):
    return  max_dv_dinv_inverse * (G.degree(node_v) -1) * (G.degree(node_inv)-1)

In [6]:
%%time
if creating_pref_attach_graph :
    print("Creating")
    G2 = networkx.Graph()
    G2.add_nodes_from(G)
    i = 0
    for v_i in vs :
        if not i%1000 :
            print(i)
        i+=1
        for inv_j in invs :
            G2.add_edge(v_i, inv_j, weight = metric_pref_attach(v_i, inv_j, G, max_dv_dinv_inverse))
    pickle.dump( G2, open( "output_data/pref_attach_edges_graph.p", "wb" ) )
else :
    print("Loading G2")
    with open('output_data/pref_attach_edges_graph.p', 'rb') as f:
        G2 = pickle.load(f)

Loading G2
CPU times: user 13.9 s, sys: 25.3 s, total: 39.2 s
Wall time: 39.3 s


In [25]:
existing_investment_edges = set(G.edges)
all_possible_edges = set(G2.edges)
all_not_existing_investment_edges = all_possible_edges - existing_investment_edges
if G4 :
    all_not_existing_investment_edges_existing_in_G4 =  set(G4.edges) - existing_investment_edges
print("Number of investment edges : ",len(existing_investment_edges))
print("Number of non-invested edges : ", len(all_not_existing_investment_edges))
print("Theoretically, possible edges are Nv x Ninv = ", len(vs)*len(invs))
print("Matching : ", len(existing_investment_edges) + len(all_not_existing_investment_edges) == len(vs)*len(invs))


Number of investment edges :  16424
Number of non-invested edges :  23121415
Theoretically, possible edges are Nv x Ninv =  23137839
Matching :  True


In [8]:
%%time
if creating_pref_attach_reduced_graph :
    print("Creating")
    G3 = networkx.Graph()
    G3.add_nodes_from(G)
    i = 0
    for v_i in vs :
        if not i%200 :
            print(i)
        i+=1
        for inv_j in invs :
            if (v_i,inv_j) in existing_investment_edges :
                G3.add_edge(v_i, inv_j, weight = metric_pref_attach_reduced(v_i, inv_j, G, max_dv_dinv_inverse))
            else : 
                G3.add_edge(v_i, inv_j, weight = metric_pref_attach(v_i, inv_j, G, max_dv_dinv_inverse))
    pickle.dump( G3, open( "output_data/pref_attach_reduced_edges_graph.p", "wb" ) )
else :
    print("Loading G3")
    with open('output_data/pref_attach_reduced_edges_graph.p', 'rb') as f:
        G3 = pickle.load(f)

Loading G3
CPU times: user 17.1 s, sys: 1min 9s, total: 1min 26s
Wall time: 1min 27s


In [9]:
sum_of_existing_edges_pref_value = 0
sum_of_non_existing_edges_pref_value = 0
for v, inv in existing_investment_edges :
    sum_of_existing_edges_pref_value += G2[v][inv]['weight']
for v, inv in all_not_existing_investment_edges :
    sum_of_non_existing_edges_pref_value += G2[v][inv]['weight']
mean_existing_pref = sum_of_existing_edges_pref_value / len(existing_investment_edges)
mean_non_existing_pref = sum_of_non_existing_edges_pref_value / len(all_not_existing_investment_edges)
print(("When a link exists, preferential attachement is on average {0}\n and {1} "
      + "when not existing").format(mean_existing_pref, mean_non_existing_pref))

When a link exists, preferential attachement is on average 8.037530164718683e-06
 and 4.985113415600712e-07 when not existing


In [10]:
sum_of_existing_edges_pref_value = 0
sum_of_non_existing_edges_pref_value = 0
for v, inv in existing_investment_edges :
    sum_of_existing_edges_pref_value += G3[v][inv]['weight']
for v, inv in all_not_existing_investment_edges :
    sum_of_non_existing_edges_pref_value += G3[v][inv]['weight']
mean_existing_pref = sum_of_existing_edges_pref_value / len(existing_investment_edges)
mean_non_existing_pref = sum_of_non_existing_edges_pref_value / len(all_not_existing_investment_edges)
print(("Method #2 : \nWhen a link exists, preferential attachement is on average {0}\n and {1} "
      + "when not existing").format(mean_existing_pref, mean_non_existing_pref))

Method #2 : 
When a link exists, preferential attachement is on average 6.823317708936446e-06
 and 4.985113415600712e-07 when not existing


In [11]:
6.82331770893643e-06 / 4.985113415602395e-07

13.687387106541706

### 3 - Measuring the connection between two nodes 

* Hitting time or Commute time (based on random walks).
* Personalized Pagerank.
* G4 - Shortest path length (or geodesic distance) Note that in weighted graph u can use **Djikstra**

In [ ]:
%%time
from copy import deepcopy
Gtemp = deepcopy(G)
if True :
    print("Creating shortest paths")
    G4 = networkx.Graph()
    G4.add_nodes_from(G)
    i = 0
    for v_i in vs :
        if i== 100:
            break;
        if not i%100 :
            print(i)
        i+=1
        for inv_j in invs :
            stored_edge = False
            if (v_i, inv_j) in existing_investment_edges :
                stored_edge = True
                Gtemp.remove_edge(v_i,inv_j)
            try :
                G4.add_edge(v_i, inv_j, weight = networkx.shortest_path_length(Gtemp, v_i, inv_j))
                Gtemp.add_edge(v_i,inv_j)
            except :
                Gtemp.add_edge(v_i,inv_j)
    pickle.dump( G4, open( "output_data/creating_shortest_paths_graph.p", "wb" ) )
    del Gtemp
else :
    print("Loading G4")
    with open('output_data/creating_shortest_paths_graph.p', 'rb') as f:
        G4 = pickle.load(f)

Creating shortest paths
0


In [31]:
%%time
sum_of_existing_edges_shortest_path = 0
sum_of_non_existing_edges_shortest_path = 0
for v, inv in existing_investment_edges :
    sum_of_existing_edges_shortest_path += G4[v][inv]['weight']
for v, inv in all_not_existing_investment_edges_existing_in_G4 :
    sum_of_non_existing_edges_shortest_path += G4[v][inv]['weight']
mean_existing_shortest_path = sum_of_existing_edges_shortest_path / len(existing_investment_edges)
mean_non_existing_shortest_path = sum_of_non_existing_edges_shortest_path / len(all_not_existing_investment_edges)
print(("Method #2 : \nWhen a link exists, preferential attachement is on average {0}\n and {1} "
      + "when not existing").format(mean_existing_shortest_path, mean_non_existing_shortest_path))

Method #2 : 
When a link exists, preferential attachement is on average 1.0
 and 2.3560139377282923 when not existing
CPU times: user 34.7 s, sys: 0 ns, total: 34.7 s
Wall time: 34.7 s


In [35]:
v, inv = list(existing_investment_edges)[2]
G4[v][inv]['weight']

1